In [ ]:
import numpy as np 
import pandas as pd 
import os
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings("ignore")

In [ ]:
hist_trans = pd.read_csv("../data/historical_transactions.csv")

In [ ]:
hist_trans.columns.values

In [ ]:
merchants = pd.read_csv("../data/merchants.csv")
dictionary = pd.read_excel("../data/Data_Dictionary.xlsx", sheet_name="merchant")

In [ ]:
dictionary

In [ ]:
merchants.head(2)

In [ ]:
merchants["category_1"] = merchants["category_1"].map({"Y": 0, "N": 1})
merchants.loc[merchants["category_2"].isnull(), "category_2"] = 0
merchants["category_4"] = merchants["category_4"].map({"Y": 0, "N": 1})

In [ ]:
merchants["merchant_category_id"].nunique(), merchants["merchant_group_id"].nunique()

#### numerical_1

In [ ]:
plt.hist(merchants["numerical_1"]);
plt.title("Distribution of numerical_1");

In [ ]:
np.percentile(merchants["numerical_1"], 95)

In [ ]:
plt.hist(merchants.loc[merchants["numerical_1"] < 0.1, "numerical_1"]);
plt.title("Distribution of numerical_1 less than 0.1");

In [ ]:
min_n1 = merchants["numerical_1"].min()
_ = sum(merchants["numerical_1"] == min_n1) / merchants["numerical_1"].shape[0]
print(f"{_ * 100: .4f} % of values in numerical_1 are equal to {min_n1}")

#### numerical_2

In [ ]:
plt.hist(merchants["numerical_2"]);
plt.title("Distribution of numerical_2");

In [ ]:
plt.hist(merchants.loc[merchants["numerical_2"] < 0.1, "numerical_2"]);
plt.title("Distribution of numerical_2 less than 0.1");
min_n2 = merchants["numerical_2"].min()
min_n2_= sum(merchants["numerical_2"] == min_n2) / merchants["numerical_2"].shape[0]
print(f"{min_n2_ *100: .4f} % of values in numerical_2 are equal to {min_n2}")
    

In [ ]:
(merchants["numerical_1"] != merchants["numerical_2"]).sum() / merchants.shape[0]

In [ ]:
merchants["most_recent_sales_range"].value_counts().plot("bar");

In [ ]:
mrsr_sort = merchants["most_recent_sales_range"].value_counts().sort_index()
gby = merchants.loc[merchants["numerical_2"] < 0.1].groupby("most_recent_sales_range")["numerical_1"].mean()       
data = [go.Bar(x=mrsr_sort.index, y=mrsr_sort.values, name="counts"), go.Scatter(x=gby.index, y=gby.values, name="mean numerical_1", yaxis="y2")]     
layout = go.Layout(dict(title="Counts of values in categories of most_recent_sales_range",
                        xaxis=dict(title="most_recent_sales_range"),
                        yaxis=dict(title="Counts"),
                        yaxis2=dict(title="mean numerical_1", overlaying="y", side="right")),
                  legend=dict(orientation="v"))
py.iplot(dict(data=data, layout=layout))



In [ ]:
merchants["most_recent_purchases_range"].value_counts().plot("bar");

In [ ]:
d = merchants['most_recent_purchases_range'].value_counts().sort_index()
e = merchants.loc[merchants['numerical_2'] < 0.1].groupby('most_recent_purchases_range')['numerical_1'].mean()
data = [go.Bar(x=d.index, y=d.values, name='counts'), go.Scatter(x=e.index, y=e.values, name='mean numerical_1', yaxis='y2')]
layout = go.Layout(dict(title = "Counts of values in categories of most_recent_purchases_range",
                        xaxis = dict(title = 'most_recent_purchases_range'),
                        yaxis = dict(title = 'Counts'),
                        yaxis2=dict(title='mean numerical_1', overlaying='y', side='right')),
                   legend=dict(orientation="v"))
py.iplot(dict(data=data, layout=layout))

#### avg_sales_lag

In [ ]:
plt.hist(merchants["avg_sales_lag3"].fillna(0));
plt.hist(merchants["avg_sales_lag6"].fillna(0));
plt.hist(merchants["avg_sales_lag12"].fillna(0));

In [ ]:
for col in ["avg_sales_lag3", "avg_sales_lag6", "avg_sales_lag12"]:
    print(f"Max value of {col} is {merchants[col].max()}")
    print(f"Min value of {col} is {merchants[col].min()}")

In [ ]:
plt.hist(merchants.loc[(merchants["avg_sales_lag12"] < 3) & (merchants["avg_sales_lag12"] > -10), 
                       "avg_sales_lag12"].fillna(0), label="avg_sales_lag12");
plt.hist(merchants.loc[(merchants["avg_sales_lag6"] < 3) & (merchants["avg_sales_lag6"] > -10),
                       "avg_sales_lag6"].fillna(0), label="avg_sales_lag6");
plt.hist(merchants.loc[(merchants["avg_sales_lag3"] < 3) & (merchants["avg_sales_lag3"] > -10),
                       "avg_sales_lag3"].fillna(0), label="avg_sales_lag3");
plt.legend();

#### avg_putchases_lag

In [ ]:
merchants["avg_purchases_lag3"].nlargest()

In [ ]:
merchants.loc[merchants["avg_purchases_lag3"] == np.inf, "avg_purchases_lag3"] = 6000
merchants.loc[merchants["avg_purchases_lag6"] == np.inf, "avg_purchases_lag6"] = 6000
merchants.loc[merchants["avg_purchases_lag12"] == np.inf, "avg_purchases_lag12"] = 6000

In [ ]:
plt.hist(merchants["avg_purchases_lag3"].fillna(0));
plt.hist(merchants["avg_purchases_lag6"].fillna(0));
plt.hist(merchants["avg_purchases_lag12"].fillna(0));

In [ ]:
plt.hist(merchants.loc[(merchants["avg_purchases_lag12"] < 4), "avg_purchases_lag12"].fillna(0),
         label="avg_purchases_lag12");
plt.hist(merchants.loc[(merchants["avg_purchases_lag6"] < 4), "avg_purchases_lag6"].fillna(0),
         label="avg_purchases_lag6");
plt.hist(merchants.loc[(merchants["avg_purchases_lag3"] < 4), "avg_purchases_lag3"].fillna(0),
         label="avg_purchases_lag3");

### 对train和trans分析

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("../data/train.csv")

In [ ]:
new_trans = pd.read_csv("../data/new_merchant_transactions.csv")

In [ ]:
train.head(2)

In [ ]:
new_trans.head(2)

In [ ]:
new_trans_train = pd.merge(train, new_trans, on="card_id", how="left")

In [ ]:
new_trans_train.shape

In [ ]:
n = new_trans_train
gby = n[(n['first_active_month']>n['purchase_date'])].groupby(["card_id"],as_index=False)

In [ ]:
train_first_month = pd.DataFrame()
train_first_month["card_id"] = train["card_id"]
train_first_month["first_active_month"] = train["first_active_month"]

In [ ]:
new_trans_train = pd.merge(train_first_month, new_trans, on="card_id", how="left")

In [ ]:
n = new_trans_train
gby = n[(n['first_active_month']>n['purchase_date'])].groupby(["card_id"],as_index=False)

In [ ]:
import datetime

n["year"] = n["first_active_month"].dt.year
n["month"] = n["first_active_month"].dt.month
n["day"] = n["first_active_month"].dt.day
n["dayofyear"] = n["first_active_month"].dt.dayofyear
n['week'] = n["first_active_month"].dt.weekofyear
n['dayofweek'] = n['first_active_month'].dt.dayofweek
n['days'] = (datetime.n(2018, 2, 1) - n['first_active_month'].dt.date).dt.days

n["authorized_flag"] = n["authorized_flag"].map({"Y": 1, "N": 0})
n["category_1"] = n["category_1"].map({"Y": 1, "N":0})
n["purchase_date"] = pd.to_datetime(n["purchase_date"])
n["year"] = n["purchase_date"].dt.year
n["month"] = n["purchase_date"].dt.month
n["day"] = n["purchase_date"].dt.day
n["weekofyear"] = n["purchase_date"].dt.weekofyear
n["dayofweek"] = n["purchase_date"].dt.dayofweek
n["weekend"] = (n["purchase_date"].dt.weekday >= 5).astype(int)
n["hour"] = n["purchase_date"].dt.hour
n["minute"] = n["purchase_date"].dt.minute
n["month_diff"] = ((datetime.datetime.today() - n["purchase_date"]).dt.days) // 30
n["month_diff"] += n["month_lag"]
n["category_2"] = n["category_2"].fillna(2.0)
n["category_3"] = n["category_3"].fillna("A")
n["merchant_id"] = n["merchant_id"].fillna("M_ID_00a6ca8a8a")

In [ ]:
agg_func = {"purchase_date":["max", "min"],
                "month_diff": ["mean"],
                "weekend": ["sum", "mean"],
                "authorized_flag": ["sum", "mean"],
                "category_1": ["sum", "mean"],
                "installments": ["sum", "max", "min", "mean", "std", "median", "var"],
                "purchase_amount": ["sum", "max", "min", "mean", "std", "median", "var"],
                "month_lag": ["max", "min", "mean", "var"],
                "month_diff": ["mean"],
                "card_id": ["size"],
                "month": ["nunique", "count"],
                "hour": ["nunique", "count"],
                "weekofyear": ["nunique", "count"],
                "dayofweek": ["nunique", "count"],
                "year": ["nunique", "count"],
                "subsector_id": ["nunique", "count"],
                "merchant_category_id": ["nunique", "count"],
                "Valentine’s_day_2017": ["mean"],
                "Christmas_day_2017": ["mean"],
                "fathers_day_2017": ["mean"],
                "Children_day_2017": ["mean"],
                "Global_shopping_2017": ["mean"],
                "Black_Friday_2017": ["mean"],
                "Valentine_day_2017": ["mean"],
                "New_year_2018": ["mean"],
                "Mothers_day_2018": ["mean"]}

In [ ]:
agg_trans = gby.agg(agg_func)